In [1]:
import random
import numpy as np
import math

In [235]:
def dex_to_bin(d):
    return bin(d).format(1)[2:].zfill(number_of_binary_positions)

def bin_to_dex(b):
    return int(b, 2)

In [234]:
number_of_genotypes = 4
number_of_binary_positions = 5 #integers are in range 0-31, where max is 32 and in binary it takes 5 bits: 11111
phenotypes = [13, 24, 8, 19]
#phenotypes = random.sample(range(0, 31), number_of_genotypes)

print "Initial population:", phenotypes

genotypes = []

for ph in phenotypes:
   b = dex_to_bin(ph) 
   genotypes.append(b)
print "Corresponding genotypes:", genotypes


Initial population: [13, 24, 8, 19]
Corresponding genotypes: ['01101', '11000', '01000', '10011']


<span style="color:green">Fitness</span>
<br>
Calculate fitness values with fitness function <b>f(x) = x<sup>2</sup></b>

In [3]:
def fitness_function(x):
    return x**2

def sum_avg_max(lov):
    s = sum(lov)
    a = round(np.mean(lov), 2)
    m = max(lov)
    
    return s, a, m

fitness_values = map(lambda x: fitness_function(x), phenotypes)
print "Fitness:", fitness_values

fitness_sum, fitness_avg, fitness_max = sum_avg_max(fitness_values)

print "Sum:", fitness_sum
print "Avg:", int(math.ceil(fitness_avg))
print "Max:", fitness_max


Fitness: [169, 576, 64, 361]
Sum: 1170
Avg: 293
Max: 576


<span style="color:green">Probability</span>

In [14]:
def probability(lov, sum_val):
    return map(lambda x: round(x/float(sum_val), 3), lov)

probability_list = probability(fitness_values, fitness_sum)
print "Probability:", probability_list

prob_sum, prob_avg, prob_max = sum_avg_max(probability_list)

print "Sum:", prob_sum
print "Avg:", prob_avg
print "Max:", prob_max

Probability: [0.144, 0.492, 0.055, 0.309]
Sum: 1.0
Avg: 0.25
Max: 0.492


<span style="color:green">Expected count</span>

In [7]:
def expected_count(lov, fitness_avg):
    return map(lambda x: round(x/float(fitness_avg), 2), lov)

expected_list = expected_count(fitness_values, fitness_avg)
print "Expected count:", expected_list

exp_sum, exp_avg, exp_max = sum_avg_max(expected_list)

print "Sum:", exp_sum
print "Avg:", exp_avg
print "Max:", exp_max

Expected count: [0.58, 1.97, 0.22, 1.23]
Sum: 4.0
Avg: 1.0
Max: 1.97


<span style="color:green">Mating pool</span>

In [129]:
#generate genotypes using numerical distribution from probability_list
def random_genotype():
    return np.random.choice(np.array(phenotypes), p=probability_list)

phenotype_mating_pool = []
genotype_mating_pool = []


for x in xrange(number_of_genotypes):
    p = random_genotype()
    g = dex_to_bin(p)
    phenotype_mating_pool.append(p)
    genotype_mating_pool.append(g)

phenotype_mating_pool = [13, 24, 24, 19]
genotype_mating_pool = ['01101', '11000', '11000', '10011']

print "Phenotype mating pool:", phenotype_mating_pool
print "Genotype mating pool:", genotype_mating_pool


Phenotype mating pool: [13, 24, 24, 19]
Genotype mating pool: ['01101', '11000', '11000', '10011']


Convert bit-strings to a list of genes for easier manipulation

In [173]:
list_of_genes = []
pair_numbering = []
#uses for calculating pair numbers
number_of_pairs = 0.5

print "List of pairs with corresponding genes:"
for g in genotype_mating_pool:
    number_of_pairs += .5
    pair_number = int(number_of_pairs)
    pair_numbering.append(pair_number)
    print "Pair", pair_number, list(g)
    list_of_genes.append(list(g))

number_of_pairs = int(number_of_pairs)    

print "Number of pairs:", number_of_pairs

List of pairs with corresponding genes:
Pair 1 ['0', '1', '1', '0', '1']
Pair 1 ['1', '1', '0', '0', '0']
Pair 2 ['1', '1', '0', '0', '0']
Pair 2 ['1', '0', '0', '1', '1']
Number of pairs: 2


<span style="color:green">Crossover point</span>

In [142]:
#crossover_point = random.randrange(1, number_of_binary_positions-1)
crossover_point = []
#create a list with crossover point, for both parents in each pair
for x in xrange(number_of_pairs):
    cp = random.randrange(1, number_of_binary_positions-1)
    crossover_point.append(cp)
    crossover_point.append(cp)

crossover_point = [4,4,2,2]
print crossover_point

[4, 4, 2, 2]


<span style="color:green">Pairs in mating pool with crossover point</span>

In [174]:
print "Pairs with crossover point:"
for line in zip(pair_numbering, list_of_genes, crossover_point):
    print "Pair", line[0], "\b:", line[1], "\b, crossover point:", line[2]

Pairs with crossover point:
Pair 1 : ['0', '1', '1', '0', '1'] , crossover point: 4
Pair 1 : ['1', '1', '0', '0', '0'] , crossover point: 4
Pair 2 : ['1', '1', '0', '0', '0'] , crossover point: 2
Pair 2 : ['1', '0', '0', '1', '1'] , crossover point: 2


<span style="color:green">Offsprings after one-point crossover</span>

In [246]:
head_genes = []
tail_genes = []
offspring_list = [] #list of list of genes
parent_1 = True
offspring_genotypes = [] #list of genotypes
offspring_phenotypes = [] #list of phenotypes

print "Head genes:"
for i in xrange(len(list_of_genes)):
    hg = list_of_genes[i][:crossover_point[i]]
    print hg
    head_genes.append(hg)

print "Tail genes:"    
for i in xrange(len(list_of_genes)):
    tg = list_of_genes[i][crossover_point[i]:]
    print tg
    tail_genes.append(tg)

#switch tails of parents in same pair and concatenate genes into offsprings
for i in xrange(len(head_genes)):
    if parent_1:
        offspring_list.append(head_genes[i] + tail_genes[i+1])
        parent_1 = False
    else:
        offspring_list.append(head_genes[i] + tail_genes[i-1])
        parent_1 = True


for os in offspring_list:
    offspring_genotypes.append(''.join(os))
    
print "\nGenotypes after crossover:\n", offspring_genotypes

#decoding to phenotypes
for og in offspring_genotypes:
    offspring_phenotypes.append(bin_to_dex(og))
    
print "\nPhenotypes after crossover:\n", offspring_phenotypes

Head genes:
['0', '1', '1', '0']
['1', '1', '0', '0']
['1', '1']
['1', '0']
Tail genes:
['1']
['0']
['0', '0', '0']
['0', '1', '1']

Genotypes after crossover:
['01100', '11001', '11011', '10000']

Phenotypes after crossover:
[12, 25, 27, 16]
